<!--Copyright © ZOMI 适用于[License](https://github.com/Infrasys-AI/AIInfra)版权许可-->

# Linear Self-Attention



### 任务要求

实现 Linear Attention 机制，给定：
- 查询矩阵 **Q**（大小 M×d）
- 键矩阵 **K**（大小 M×d）
- 值矩阵 **V**（大小 M×d）

使用以下公式计算输出矩阵：

$$
\text{LinearAttention}(Q, K, V) = \frac{\phi(Q) \left(\phi(K)^T V\right)}{\phi(Q) \left(\sum_{j} \phi(K_j)\right)}
$$

其中 $\phi(\mathbf{x})$ 是特征映射函数（Feature Map），定义为：

$$
\phi(x) = \text{ELU}(x) + 1 =
\begin{cases}
x + 1, & x > 0 \\
e^x, & x \leq 0
\end{cases}
$$

### 约束条件

- 矩阵 **Q**、**K**、**V** 和 **output** 的数据类型均为 `float32`
- M 和 d 的数据类型为 `int32`
- 序列长度：1 ≤ M ≤ 10000
- 特征维度：1 ≤ d ≤ 128
- 矩阵元素范围：[-100.0, 100.0]

### 实现要求

1. **仅使用原生 CUDA 特性**：不允许使用外部库（如 cuBLAS）
2. **函数签名保持不变**：`solve` 函数接口必须按要求实现
3. **结果存储**：最终输出必须写入 `output` 矩阵


##  Linear Attention 核心原理

###  传统 Softmax Attention 的瓶颈

标准的 Transformer 自注意力机制的计算公式为：

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d}}\right)V
$$

**问题分析**：
- **计算复杂度**：O(M² × d)
  - 计算 $QK^T$ 需要 O(M² × d) 次操作
  - 得到的注意力矩阵大小为 M×M
- **空间复杂度**：O(M²)
  - 需要存储完整的注意力矩阵
- **扩展性瓶颈**：
  - 当 M=1024 时，注意力矩阵需要 4MB 内存
  - 当 M=8192 时，注意力矩阵需要 256MB 内存
  - 内存占用随序列长度平方增长

###  Linear Attention 的突破

Linear Attention 的核心思想是**通过核技巧（Kernel Trick）改变计算顺序**，避免显式计算 M×M 的注意力矩阵。

#### 关键洞察

观察以下两种计算顺序：

**标准 Attention（O(M²)）**：
1. 先计算 $A = \text{softmax}(QK^T)$，得到 M×M 矩阵
2. 再计算 $\text{Output} = AV$

**Linear Attention（O(M)）**：
1. 先计算 $S = \phi(K)^T V$，得到 d×d 矩阵
2. 再计算 $\text{Output} = \phi(Q) S$

关键在于：**矩阵乘法满足结合律**，通过改变括号位置，可以避免生成大矩阵。

#### 数学推导

标准 Softmax Attention 可以近似为：

$$
\text{Attention}(Q, K, V) \approx \frac{\phi(Q) \phi(K)^T V}{\text{normalizer}}
$$

其中归一化项为：

$$
\text{normalizer} = \phi(Q) \left(\sum_{j=1}^{M} \phi(K_j)\right)
$$

利用结合律重排计算顺序：

$$
\phi(Q) \left(\phi(K)^T V\right) \text{ 代替 } \left(\phi(Q) \phi(K)^T\right) V
$$

这样就避免了计算 M×M 的 $\phi(Q) \phi(K)^T$ 矩阵。

###  复杂度对比

| 操作 | Softmax Attention | Linear Attention |
|------|-------------------|------------------|
| **时间复杂度** | O(M² × d) | O(M × d²) |
| **空间复杂度** | O(M²) | O(d²) |
| **瓶颈操作** | 计算 QK^T（M×M 矩阵） | 计算 K^TV（d×d 矩阵） |
| **适用场景** | 短序列（M < 512） | 长序列（M >> d） |

**性能分析**（M=1024, d=64）：

**Softmax Attention**：
- 注意力矩阵：1024×1024 = 1,048,576 个元素
- 计算量：≈ 67M 次浮点运算
- 内存：4MB

**Linear Attention**：
- 中间矩阵：64×64 = 4,096 个元素
- 计算量：≈ 4M 次浮点运算
- 内存：16KB

**加速比**：约 **16× 时间复杂度降低**，**256× 空间复杂度降低**

###  特征映射函数 φ(x) 的选择

本实现采用 **ELU+1** 作为特征映射：

$$
\phi(x) = \text{ELU}(x) + 1 =
\begin{cases}
x + 1, & x > 0 \\
e^x, & x \leq 0
\end{cases}
$$

**设计考量**：
- **非负性**：$\phi(x) > 0$，确保注意力权重为正（类似 Softmax）
- **平滑性**：可微且连续，有利于梯度传播
- **表达能力**：能够捕捉输入的非线性关系
- **数值稳定性**：
  - 当 x > 0 时，输出为线性增长（避免指数爆炸）
  - 当 x ≤ 0 时，输出为指数衰减（保持平滑过渡）


接下来实现 CUDA 版本的 Linear Self-Attention：

In [ ]:
#include <cuda_runtime.h>
#include <cmath>

// Feature map: ELU(x) + 1
__device__ float phi(float x) {
    if (x > 0.0f) {
        return x + 1.0f;
    } else {
        return expf(x);
    }
}

// Apply phi element-wise to a matrix
__global__ void apply_phi_kernel(const float* input, float* output, int M, int d) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = M * d;

    if (idx < total) {
        output[idx] = phi(input[idx]);
    }
}

// Compute phi(K)^T * V -> result is (d x d)
__global__ void compute_KT_V_kernel(const float* phi_K, const float* V, float* KT_V, int M, int d) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < d && col < d) {
        float sum = 0.0f;
        for (int i = 0; i < M; i++) {
            sum += phi_K[i * d + row] * V[i * d + col];
        }
        KT_V[row * d + col] = sum;
    }
}

// Compute sum_j phi(K_j) -> result is (d,)
__global__ void compute_sum_phi_K_kernel(const float* phi_K, float* sum_phi_K, int M, int d) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (col < d) {
        float sum = 0.0f;
        for (int i = 0; i < M; i++) {
            sum += phi_K[i * d + col];
        }
        sum_phi_K[col] = sum;
    }
}

// Compute numerator: phi(Q) * (phi(K)^T * V) -> result is (M x d)
__global__ void compute_numerator_kernel(const float* phi_Q, const float* KT_V, float* numerator, int M, int d) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < d) {
        float sum = 0.0f;
        for (int i = 0; i < d; i++) {
            sum += phi_Q[row * d + i] * KT_V[i * d + col];
        }
        numerator[row * d + col] = sum;
    }
}

// Compute denominator: phi(Q) * sum_phi_K -> result is (M,)
__global__ void compute_denominator_kernel(const float* phi_Q, const float* sum_phi_K, float* denominator, int M, int d) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M) {
        float sum = 0.0f;
        for (int i = 0; i < d; i++) {
            sum += phi_Q[row * d + i] * sum_phi_K[i];
        }
        denominator[row] = sum;
    }
}

// Divide numerator by denominator element-wise
__global__ void divide_kernel(const float* numerator, const float* denominator, float* output, int M, int d) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < d) {
        output[row * d + col] = numerator[row * d + col] / denominator[row];
    }
}

// Q, K, V, output are device pointers
extern "C" void solve(const float* Q, const float* K, const float* V, float* output, int M, int d) {
    // Allocate temporary buffers
    float *phi_Q, *phi_K, *KT_V, *sum_phi_K, *numerator, *denominator;

    cudaMalloc(&phi_Q, M * d * sizeof(float));
    cudaMalloc(&phi_K, M * d * sizeof(float));
    cudaMalloc(&KT_V, d * d * sizeof(float));
    cudaMalloc(&sum_phi_K, d * sizeof(float));
    cudaMalloc(&numerator, M * d * sizeof(float));
    cudaMalloc(&denominator, M * sizeof(float));

    // Step 1: Apply phi to Q and K
    int total_elements = M * d;
    int threads_1d = 256;
    int blocks_1d = (total_elements + threads_1d - 1) / threads_1d;

    apply_phi_kernel<<<blocks_1d, threads_1d>>>(Q, phi_Q, M, d);
    apply_phi_kernel<<<blocks_1d, threads_1d>>>(K, phi_K, M, d);

    // Step 2: Compute phi(K)^T * V (d x d)
    dim3 threads_2d(16, 16);
    dim3 blocks_KT_V((d + 15) / 16, (d + 15) / 16);
    compute_KT_V_kernel<<<blocks_KT_V, threads_2d>>>(phi_K, V, KT_V, M, d);

    // Step 3: Compute sum_j phi(K_j)
    int blocks_sum = (d + threads_1d - 1) / threads_1d;
    compute_sum_phi_K_kernel<<<blocks_sum, threads_1d>>>(phi_K, sum_phi_K, M, d);

    // Step 4: Compute numerator phi(Q) * KT_V (M x d)
    dim3 blocks_num((d + 15) / 16, (M + 15) / 16);
    compute_numerator_kernel<<<blocks_num, threads_2d>>>(phi_Q, KT_V, numerator, M, d);

    // Step 5: Compute denominator phi(Q) * sum_phi_K (M,)
    int blocks_denom = (M + threads_1d - 1) / threads_1d;
    compute_denominator_kernel<<<blocks_denom, threads_1d>>>(phi_Q, sum_phi_K, denominator, M, d);

    // Step 6: Divide numerator by denominator
    dim3 blocks_div((d + 15) / 16, (M + 15) / 16);
    divide_kernel<<<blocks_div, threads_2d>>>(numerator, denominator, output, M, d);

    cudaDeviceSynchronize();

    // Free temporary buffers
    cudaFree(phi_Q);
    cudaFree(phi_K);
    cudaFree(KT_V);
    cudaFree(sum_phi_K);
    cudaFree(numerator);
    cudaFree(denominator);
}


本实现包含 6 个 CUDA 内核：

1. **`apply_phi_kernel`**：并行应用特征映射 φ(x)
   - 线程分配：每个线程处理一个元素
   - 复杂度：O(M×d)

2. **`compute_KT_V_kernel`**：计算 φ(K)^T V
   - 线程分配：16×16 线程块处理 d×d 输出矩阵
   - 复杂度：O(M×d²)

3. **`compute_sum_phi_K_kernel`**：计算列和
   - 线程分配：每个线程负责一列的求和
   - 复杂度：O(M×d)

4. **`compute_numerator_kernel`**：计算 φ(Q) · KT_V
   - 线程分配：16×16 线程块处理 M×d 输出矩阵
   - 复杂度：O(M×d²)

5. **`compute_denominator_kernel`**：计算 φ(Q) · sum_phi_K
   - 线程分配：每个线程处理一行的点积
   - 复杂度：O(M×d)

6. **`divide_kernel`**：逐元素相除
   - 线程分配：16×16 线程块处理 M×d 输出矩阵
   - 复杂度：O(M×d)

### 内存管理

**中间缓冲区**：
- `phi_Q`：M×d（φ(Q) 的结果）
- `phi_K`：M×d（φ(K) 的结果）
- `KT_V`：d×d（键-值交互矩阵）
- `sum_phi_K`：d（键的列和）
- `numerator`：M×d（分子）
- `denominator`：M（分母）

**内存占用估算**（以 M=1024, d=64, float32 为例）：
- `phi_Q` 和 `phi_K`：2 × 1024 × 64 × 4 bytes = 512 KB
- `KT_V`：64 × 64 × 4 bytes = 16 KB
- `sum_phi_K`：64 × 4 bytes = 256 bytes
- `numerator`：1024 × 64 × 4 bytes = 256 KB
- `denominator`：1024 × 4 bytes = 4 KB
- **总计**：≈ 788 KB

对比传统 Attention 的注意力矩阵（1024×1024×4 bytes = 4 MB），内存节省约 **80%**。
